In [1]:
import torch

print("Is cuda available?", torch.cuda.is_available())
print(torch.version.cuda)
print("Is cuDNN version:", torch.backends.cudnn.version())
print("cuDNN enabled? ", torch.backends.cudnn.enabled)

Is cuda available? True
11.7
Is cuDNN version: 8500
cuDNN enabled?  True


In [2]:
# List all available GPUs
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Perform a simple operation on the GPU
if torch.cuda.is_available():
    # Create a tensor and move it to GPU
    x = torch.tensor([1.0, 2.0]).cuda()
    y = torch.tensor([1.0, 2.0]).cuda()

    # Perform a simple operation
    z = x + y
    print("Result of the tensor operation on GPU:", z)
    print("Device of z:", z.device)
else:
    print("CUDA is not available. Operations will run on CPU.")

GPU 0: NVIDIA GeForce RTX 2080 Ti
GPU 1: NVIDIA GeForce RTX 2080 Ti
GPU 2: NVIDIA GeForce RTX 2080 Ti
GPU 3: NVIDIA GeForce RTX 2080 Ti
Result of the tensor operation on GPU: tensor([2., 4.], device='cuda:0')
Device of z: cuda:0


In [3]:
import tensorflow as tf
# Check if CUDA (GPU support) is available
print("Is CUDA available?", tf.test.is_built_with_cuda())

# Check for GPU devices
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", len(gpus), "GPUs:", gpus)

2024-04-03 10:29:33.098829: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 10:29:34.009839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Is CUDA available? True
Available GPUs: 4 GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [4]:
import tensorflow as tf

# List all available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for i, gpu in enumerate(gpus):
        print(f"GPU {i}: {gpu.name}")
else:
    print("No GPUs found, operations will run on CPU.")

# Perform a simple operation on the GPU
if gpus:
    # TensorFlow automatically uses a GPU if available for operations,
    # but you can explicitly set a device to ensure operations run on it.
    with tf.device('/GPU:0'):  # This specifies the first GPU as the target device
        # Create tensors
        x = tf.constant([1.0, 2.0])
        y = tf.constant([1.0, 2.0])

        # Perform a simple operation
        z = x + y

    print("Result of the tensor operation on GPU:", z.numpy())
    # TensorFlow's Tensor objects have a .device attribute that shows the device placement
    print("Device of z:", z.device)
else:
    print("CUDA is not available. Operations will run on CPU.")

GPU 0: /physical_device:GPU:0
GPU 1: /physical_device:GPU:1
GPU 2: /physical_device:GPU:2
GPU 3: /physical_device:GPU:3
Result of the tensor operation on GPU: [2. 4.]
Device of z: /job:localhost/replica:0/task:0/device:GPU:0


2024-04-03 10:29:35.528060: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2024-04-03 10:29:35.529328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9796 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1e:00.0, compute capability: 7.5
2024-04-03 10:29:35.530403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9796 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:89:00.0, compute capability: 7.5
2024-04-03 10:29:35.530947: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9796 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

In [5]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle"""
import deepxde as dde
import matplotlib.pyplot as plt
import numpy as np
# Import tf if using backend tensorflow.compat.v1 or tensorflow
from deepxde.backend import tf
# Import torch if using backend pytorch
# import torch
# Import jax.numpy if using backend jax
# import jax.numpy as jnp
# Import paddle if using backend paddle
# import paddle
tf.config.set_visible_devices([], 'GPU')

def pde(x, y):
    # Most backends
    dy_xx = dde.grad.hessian(y, x)
    # Backend jax
    # dy_xx, _ = dde.grad.hessian(y, x)
    # Use tf.sin for backend tensorflow.compat.v1 or tensorflow
    return -dy_xx - np.pi ** 2 * tf.sin(np.pi * x)
    # Use torch.sin for backend pytorch
    # return -dy_xx - np.pi ** 2 * torch.sin(np.pi * x)
    # Use jax.numpy.sin for backend jax
    # return -dy_xx - np.pi ** 2 * jnp.sin(np.pi * x)
    # Use paddle.sin for backend paddle
    # return -dy_xx - np.pi ** 2 * paddle.sin(np.pi * x)


def boundary(x, on_boundary):
    return on_boundary


def func(x):
    return np.sin(np.pi * x)


geom = dde.geometry.Interval(-1, 1)
bc = dde.icbc.DirichletBC(geom, func, boundary)
data = dde.data.PDE(geom, pde, bc, 16, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])

losshistory, train_state = model.train(iterations=10000)
# Optional: Save the model during training.
# checkpointer = dde.callbacks.ModelCheckpoint(
#     "model/model", verbose=1, save_better_only=True
# )
# Optional: Save the movie of the network solution during training.
# ImageMagick (https://imagemagick.org/) is required to generate the movie.
# movie = dde.callbacks.MovieDumper(
#     "model/movie", [-1], [1], period=100, save_spectrum=True, y_reference=func
# )
# losshistory, train_state = model.train(iterations=10000, callbacks=[checkpointer, movie])

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

# Optional: Restore the saved model with the smallest training loss
# model.restore(f"model/model-{train_state.best_step}.ckpt", verbose=1)
# Plot PDE residual
x = geom.uniform_points(1000, True)
y = model.predict(x, operator=pde)
plt.figure()
plt.plot(x, y)
plt.xlabel("x")
plt.ylabel("PDE residual")
plt.show()


Using backend: tensorflow
Other supported backends: tensorflow.compat.v1, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Enable just-in-time compilation with XLA.



RuntimeError: Visible devices cannot be modified after being initialized